In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import requests
from pymongo import MongoClient
import numpy as np
from collections import defaultdict
import pickle
import networkx as nx

In [2]:
from webscrape_util.scrape_util import (load_json,
                        write_json,
                        selenium_instagram_login,
                        setup_mongo_client)


In [3]:
client, collection = setup_mongo_client('instascrape', 'influencer_likes')

# Making dictionary of top posts for top influencers

In [4]:
hashtags_likes_dict = load_json('../data/hashtags_likes_dict.json')
top = nx.read_gml('../data/graph_withSelenaGomez_reduced.gml')

In [5]:
# Collect posts for all top influencers
influencers_top = {}
for k,v in hashtags_likes_dict.iteritems():
    if k in top.nodes():
        influencers_top[k] = v

In [6]:
# Collect most liked post for all top influencers
influencers_top_post = {}
for k,v in influencers_top.iteritems():
    influencers_top_post[k] = {'max_likes':0, 'post': None}
    for i in range(len(v['likes'])):
        if v['likes'][i] > influencers_top_post[k]['max_likes']:
            influencers_top_post[k]['max_likes'] = v['likes'][i]
            influencers_top_post[k]['post'] = v['posts'][i]

In [7]:
top = influencers_top_post.keys()

# SCRAPE THE REST!

In [8]:
driver = webdriver.Firefox()
url_likes = "https://www.instagram.com/graphql/query/?query_id=17864450716183058&variables=%7B%22shortcode%22%3A%22{}%22%2C%22first%22%3A{}%7D"
selenium_instagram_login(driver, '../instagram_credentials.json')

In [9]:
for i in range(len(top)):
    driver.get(url_likes.format(influencers_top_post[top[i]]['post'], str(influencers_top_post[top[i]]['max_likes'])))
    time.sleep(np.random.uniform(3,5))
    
    driver.find_element_by_id('tab-1').click()
    data = driver.find_element_by_css_selector('pre.data')
    my_json = json.loads(data.text)
    likes = my_json['data']['shortcode_media']['edge_liked_by']['edges']
    collection.insert_one({'id':top[i],'likes': likes})
    
    time.sleep(np.random.uniform(5,8))

# Saving the collected data

In [10]:
cursor = collection.find({})
for x in cursor:
    user_id = x['id']
    likes = []
    for liker in x['likes']:
        likes.append(liker['node']['id'])
    if influencers_top_post.get(user_id, False):
        influencers_top_post[user_id]['likes'] = likes

## Add total followers

In [11]:
G = nx.read_gml('../data/file.gml')

In [12]:
for influencer in influencers_top_post:
    influencers_top_post[influencer]['num_follow'] = len(G.in_edges(influencer))

In [13]:
write_json(influencers_top_post, '../data/interaction_dict.json')

# Checking the data

In [14]:
for k,v in influencers_top_post.iteritems():
    if 'likes' in v.keys():
        print k, v['max_likes'], len(v['likes'])